# Load Data

In [6]:
from pyrecdp.datasets import ibm_fraud_detect

full_data = ibm_fraud_detect().to_pandas()
train_data = full_data[full_data['Year'] <= 2018]
train_data

,User,Card,Year,Month,Day,Time,Amount,Use Chip,Merchant Name,Merchant City,Merchant State,Zip,MCC,Errors?,Is Fraud?
0,0,0,2002,9,1,06:21,$134.09,Swipe Transaction,3527213246127876953,La Verne,CA,91750.0,5300,NaN,No
1,0,0,2002,9,1,06:42,$38.48,Swipe Transaction,-727612092139916043,Monterey Park,CA,91754.0,5411,NaN,No
2,0,0,2002,9,2,06:22,$120.34,Swipe Transaction,-727612092139916043,Monterey Park,CA,91754.0,5411,NaN,No
3,0,0,2002,9,2,17:45,$128.95,Swipe Transaction,3414527459579106770,Monterey Park,CA,91754.0,5651,NaN,No
4,0,0,2002,9,3,06:23,$104.71,Swipe Transaction,5817218446178736267,La Verne,CA,91750.0,5912,NaN,No
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24385063,1999,1,2018,12,30,11:42,$50.21,Chip Transaction,-3345936507911876459,Manchester,NH,3102.0,7538,NaN,No
24385064,1999,1,2018,12,30,20:17,$46.29,Online Transaction,-5841929396161652653,ONLINE,NaN,NaN,4121,NaN,No
24385065,1999,1,2018,12,31,09:20,$5.11,Online Transaction,-7421093378627544099,ONLINE,NaN,NaN,5311,NaN,No
24385066,1999,1,2018,12,31,11:28,$77.22,Chip Transaction,2910328604019547969,Milford,NH,3055.0,7538,NaN,No


# RecDP Analyze data